## Conectar ao SQLite

In [3]:
import sqlite3
import pandas as pd

In [4]:
conn = sqlite3.connect("../database/dados_tratados.db")

In [6]:
pd.read_sql(
    "SELECT name FROM sqlite_master WHERE type='table';",
    conn
)

,name
0,vendas_tratadas
1,produtos_tratados
2,clientes_tratados
3,metas_tratadas


In [8]:
vendas = pd.read_sql("SELECT * FROM vendas_tratadas", conn)
produtos = pd.read_sql("SELECT * FROM produtos_tratados", conn)
clientes = pd.read_sql("SELECT * FROM clientes_tratados", conn)
metas = pd.read_sql("SELECT * FROM metas_tratadas", conn)

In [9]:
vendas['data'] = pd.to_datetime(vendas['data'])

## Criar base analítica 

In [10]:
df = vendas.merge(produtos, on="id_produto", how="left")
df = df.merge(clientes, on="id_cliente", how="left")

In [11]:
df['lucro_unitario'] = df['valor_unitario'] - df['custo_unitario']
df['lucro_total'] = df['quantidade'] * df['lucro_unitario']
df['margem'] = df['lucro_total'] / df['valor_total']

In [12]:
df['ano_mes'] = df['data'].dt.to_period('M')

## Receita

In [13]:
receita_total = df['valor_total'].sum()
receita_mensal = df.groupby('ano_mes')['valor_total'].sum().reset_index()
receita_mensal['crescimento_%'] = receita_mensal['valor_total'].pct_change() * 100

## Lucro

In [14]:
lucro_total = df['lucro_total'].sum()
lucro_mensal = df.groupby('ano_mes')['lucro_total'].sum().reset_index()
lucro_mensal['crescimento_%'] = lucro_mensal['lucro_total'].pct_change() * 100

## Margem

In [15]:
margem_media = df['lucro_total'].sum() / df['valor_total'].sum()

In [16]:
margem_mensal = (
    df.groupby('ano_mes')
      .agg({'lucro_total':'sum', 'valor_total':'sum'})
      .reset_index()
)

margem_mensal['margem'] = (
    margem_mensal['lucro_total'] / margem_mensal['valor_total']
)

## Meta vs Realizado

In [18]:
metas.columns 

Index(['mes', 'meta_faturamento'], dtype='object')

In [19]:
metas['ano_mes'] = pd.to_datetime(metas['mes']).dt.to_period('M')

In [21]:
metas.dtypes

mes                    object
meta_faturamento        int64
ano_mes             period[M]
dtype: object

In [22]:
metas['mes'] = pd.to_datetime(metas['mes'])

In [24]:
meta_vs_real = receita_mensal.merge(metas, on='ano_mes', how='left')

meta_vs_real['dif_abs'] = (
    meta_vs_real['valor_total'] - meta_vs_real['meta_faturamento']
)

meta_vs_real['dif_%'] = (
    meta_vs_real['valor_total'] / meta_vs_real['meta_faturamento'] - 1
) * 100

## Análisar se atingiu a meta

In [25]:
meta_vs_real['status_meta'] = meta_vs_real['dif_abs'].apply(
    lambda x: 'Acima da Meta' if x >= 0 else 'Abaixo da Meta'
)

meta_vs_real

,ano_mes,valor_total,crescimento_%,mes,meta_faturamento,dif_abs,dif_%,status_meta
0,2024-01,68111.7,NaN,2024-01-01,59725,8386.7,14.042193,Acima da Meta
1,2024-02,41143.1,-39.594666,2024-02-01,55353,-14209.9,-25.671418,Abaixo da Meta
2,2024-03,57032.1,38.618869,2024-03-01,38115,18917.1,49.631641,Acima da Meta
3,2024-04,102386.8,79.524864,2024-04-01,35387,66999.8,189.334501,Acima da Meta
4,2024-05,76559.9,-25.224834,2024-05-01,53919,22640.9,41.990578,Acima da Meta
5,2024-06,127007.2,65.892589,2024-06-01,59945,67062.2,111.872883,Acima da Meta


In [27]:
vendas.columns 

Index(['data', 'id_produto', 'id_cliente', 'canal_venda', 'quantidade',
       'valor_unitario', 'valor_total'],
      dtype='object')

In [26]:
vendas.groupby('ano_mes')['valor_total'].sum().sort_index()

KeyError: 'ano_mes'